In [2]:
from bitarray import bitarray
import pickle
import numpy as np
import torch

In [25]:
def as_bit_tensor(tensor, num_bits=2):
    """Cast a tensor of int type into binary
    """
    shape = tensor.shape
    array = tensor.reshape(-1).numpy()
    bool_arrays = [np.zeros_like(array, dtype=np.bool) for _ in range(num_bits)]     # construct a list of boolean array
    for i, bit in enumerate(reversed(range(num_bits))):
        idx = array >= (2**bit)
        bool_arrays[i][idx] = True
        array = array - (2**bit) * idx
    bit_arrays = [bitarray(bool_ten.tolist()) for bool_ten in bool_arrays]
    
    return {'s': shape, 'd': bit_arrays}


def as_int_tensor(bit_tensor):
    tensor = torch.zeros(bit_tensor['s'], dtype=torch.int)
    for i, bt in enumerate(reversed(bit_tensor['d'])):
        tensor += (2 ** i) * torch.tensor(bt.tolist(), dtype=torch.bool).reshape(bit_tensor['s'])
    return tensor


In [31]:
a = torch.randint(4, (100, 1024))
b = as_bit_tensor(a, num_bits=2)
c = as_int_tensor(b)
print(torch.sum(torch.abs(a-c)))

tensor(0)


In [32]:
pickle.dump(a, open('origin_tensor', 'wb')) #  800.4 KB
pickle.dump(b, open('bit_tensor', 'wb'))    #  25.1 KB